In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Crea (o abre si ya existe) una base de datos llamada "mi_base.db".
conn = sqlite3.connect("mi_base.db")
# Crea un cursor para ejecutar comandos SQL.
cursor = conn.cursor()

Ejemplo 1

In [ ]:
# Ejemplo 1.
cursor.execute("""
SELECT * FROM usuarios
LIMIT 10;
""")
print(cursor.fetchall())

In [ ]:
# Visualizamos con pandas.
cursor.execute("""
SELECT * FROM usuarios
LIMIT 10;
""")

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=columns)

In [ ]:
df.head()

In [ ]:
# Cerrar conexion.
conn.close()

Ejemplo 2

In [ ]:
# Ejemplo 2.

# 1. Conexión a la base de datos.
conn = sqlite3.connect("mi_base.db")
cursor = conn.cursor()

# 2. Ejecutar query: traer toda la tabla ordenes.
query = "SELECT * FROM ordenes;"
cursor.execute(query)

# 3. Convertir a DataFrame.
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [ ]:
# 4. Convertir la columna 'fecha' a tipo datetime.
df['fecha'] = pd.to_datetime(df['fecha'])
# 5. Crear columna "mes-año".
df['mes_año'] = df['fecha'].dt.to_period('M').astype(str)
# 6. Agrupar por mes-año y sumar montos.
ventas_mensuales = df.groupby('mes_año')['monto'].sum().reset_index()
# 7. Ordenar por fecha real.
ventas_mensuales['mes_año'] = pd.to_datetime(ventas_mensuales['mes_año'])
ventas_mensuales = ventas_mensuales.sort_values('mes_año')

In [ ]:
df.head()

In [ ]:
# Creamos carpeta de outputs y guardamos.
df.to_csv(r".\output\bbdd\df_ventas.csv", index=False)

In [ ]:
# 8. Graficar.
plt.figure(figsize=(10, 5))
plt.plot(ventas_mensuales['mes_año'], ventas_mensuales['monto'], marker='o')
plt.title('Evolución de ventas mensuales')
plt.xlabel('Mes-Año')
plt.ylabel('Total de ventas (S/)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
# Guardar grafico.
plt.savefig(r".\output\bi\ventas_mensuales.png", dpi=300, bbox_inches='tight')
# Mostrar grafico.
plt.show()

In [ ]:
# 9. Cerrar conexión.
conn.close()

Ejemplo 3

In [ ]:
# Ejemplo 3: usuarios registrados en 2025.

# Conexión.
conn = sqlite3.connect("mi_base.db")
cursor = conn.cursor()

# Query: usuarios registrados en 2025.
cursor.execute("""
SELECT nombre, email, fecha_registro
FROM usuarios
WHERE fecha_registro LIKE '2025%';
""")

# Convertir a DataFrame.
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df_usuarios_2025 = pd.DataFrame(rows, columns=columns)

# Mostrar resultado.
print(df_usuarios_2025)

conn.close()

Ejemplo 4

In [ ]:
# Últimas 10 órdenes con nombre y correo del usuario.

# Conexión.
conn = sqlite3.connect("mi_base.db")
cursor = conn.cursor()

# Query: join de usuarios + órdenes (últimos 10).
cursor.execute("""
SELECT u.nombre, u.email, o.producto, o.monto, o.fecha
FROM ordenes o
JOIN usuarios u ON o.usuario_id = u.id
ORDER BY o.fecha DESC
LIMIT 10;
""")

# Convertir a DataFrame.
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df_ordenes_recientes = pd.DataFrame(rows, columns=columns)

# Mostrar resultado.
print(df_ordenes_recientes)

conn.close()